# Helmes Hackday machine learning tutorial
### Email spam detection using Python and scikit-learn



TODO some introductory text about the task at hand and the approach we will take. spam vs ham

Let's start with some imports. The most relevant to machine learning here are **numpy** -- a very useful matrix computation library -- and **sklearn**, a standard library for training machine learning problems.

In [1]:
import numpy as np
import glob
import codecs
import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn import grid_search
from sklearn import preprocessing

Let's seed the pseudo-random generator so the results won't change on each run.

In [2]:
np.random.seed(42)

Like in all machine learning problems, we need to have some sort of numeric representation for our emails -- most machine learning models take numeric vectors as input. Let's write a very simple function for extracting some numbers (called features) from an email.

These features are a reasonable guess -- spam often contains weird characters and a lot of capital letters but in principle we could plug in any numbers. If you think it could be useful, it's safer to use rather than omit it.

In [7]:
def simple_features(text):
    """Extract some simple numeric features from one email."""
    length = len(text)
    proportion_upper_case = sum(1 for c in text if c.isupper()) / length
    proportion_lower_case = sum(1 for c in text if c.islower()) / length
    proportion_alphanum = sum(1 for c in text if c.isalnum()) / length
    return [length, proportion_upper_case, proportion_lower_case, proportion_alphanum]

Our dataset is scattered in a bunch of files, one for each email, however we would like to have a matrix of numbers where each row vector corresponds to the features we extracted from one email. Note that `feature_extraction_function` can be any function that returns a list of numbers, e.g. `stupid_features()`.

We also need labels telling us which vectors corresponds to spam emails. Let's say we want to flag spam so spam corresponds to the label '1' and ham corresponds to '0'.

In [11]:
def get_dataset(feature_extraction_function, number_of_emails=5):
    """Read all ham and spam emails from files and return features and labels."""

    # Data downloaded from
    # http://www.aueb.gr/users/ion/data/enron-spam/

    ham_pattern  = "data/enron*/ham/*.txt"
    spam_pattern = "data/enron*/spam/*.txt"

    # Initialise
    features = []
    labels   = []

    # Find all files containing ham and spam emails and concatenate them with the corresponding label
    ham_files = glob.glob(ham_pattern)
    ham_files = ham_files[0:min(number_of_emails, len(ham_files))]
    spam_files = glob.glob(spam_pattern)
    spam_files = spam_files[0:min(number_of_emails, len(spam_files))]
    files = list(zip(ham_files, [0] * len(ham_files))) + list(zip(spam_files, [1] * len(spam_files)))

    for filename, label in files:
        with codecs.open(filename, "r", encoding="utf-8", errors="ignore") as f:
            text = "".join(f.readlines())
            email_features = feature_extraction_function(text)
            labels.append(label)              # Last entry in feature vec

            features.append(email_features)

    # Convert to Numpy objects because that's what scikit-learn expects
    features = np.asarray(features)
    labels   = np.asarray(labels)

    print("Extracted features using function '%s'" % (feature_extraction_function.__name__))
    print("%d emails processed: %d ham, %d spam" % (len(features), len(ham_files), len(spam_files)))

    return features, labels

OK, let's get our dataset and check what the first five rows look like.

In [13]:
NUM_EMAILS = 1000  # How many emails each of ham & spam do we want to use?
features, labels = get_dataset(simple_features, number_of_emails=NUM_EMAILS)
print(features[0:5,:])
print(labels[0:5])

Extracted features using function 'simple_features'
2000 emails processed: 1000 ham, 1000 spam
[[  3.90000000e+01   2.56410256e-02   7.94871795e-01   8.20512821e-01]
 [  4.42900000e+03   2.25784601e-04   2.88326936e-01   3.59674870e-01]
 [  7.70000000e+01   1.29870130e-02   7.66233766e-01   7.92207792e-01]
 [  1.21900000e+03   8.20344545e-04   4.95488105e-01   5.52091879e-01]
 [  1.18300000e+03   8.45308538e-04   5.27472527e-01   5.78191040e-01]]
[0 0 0 0 0]


Now that we have our data, it's time to do some machine learning. Let's start with a very simple model: [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression) which is like linear regression (fitting a straight line through your data) with a small additional step to turn real numbers into 0/1 classification decisions.

In [14]:
print("\n---- FIRST MODEL: LOGISTIC REGRESSION ----")
model1 = LogisticRegression()
model1.fit(features, labels)
predicted_labels = model1.predict(features)
training_accuracy = sklearn.metrics.accuracy_score(labels, predicted_labels)
print("Accuracy on training set: %.3f" % training_accuracy)


---- FIRST MODEL: LOGISTIC REGRESSION ----
Accuracy on training set: 0.684


That's it! What did we do?
* initialised a logistic regression model,
* told it to train itself (fit itself to the data),
* asked the model what it would predict on the same data it trained on,
* calculated the accuracy, i.e. percentage of correct guesses.

## Accuracy, precision and recall

Accuracy might not always be a good measure of how good a model is. In cancer detection, perhaps 0.01% of anyone you test actually has cancer and the rest are healthy. If you maximise accuracy, you can just say 'no-one has cancer' and your accuracy is 99.99%. In all cases where classes are imbalanced (i.e. the split isn't 50-50 cancer vs healthy) we want to use something more sophisticated.

By the way, when we compare our predictions with the ground truth, we get four cases:
* True Positive (TP): we correctly classified an email as spam.
* True Negative (TN): we correctly classified an email as not spam.
* False Positive (FP): we said the email was spam but it was actually ham.
* False Negative (FN): we said the email was ham but it was actually spam.

<img src="http://3.bp.blogspot.com/_txFWHHNYMJQ/THyADzbutYI/AAAAAAAAAf8/TAXL7lySrko/s1600/Picture+8.png" alt="Confusion matrix" style="height: 200px;"/>

This is why we use [**precision** and **recall**](https://en.wikipedia.org/wiki/Precision_and_recall). In our case, precision is the percentage of emails that we correctly classified as spam -- TP / (TP + FP) -- and recall is the percentage of spam emails that we correctly flagged as spam out of all emails that actually were spam -- TP / (TP + FN).

It takes a while before these terms become more intuitive so keep the definitions handy.

In [15]:
training_precision = sklearn.metrics.precision_score(labels, predicted_labels)
training_recall = sklearn.metrics.recall_score(labels, predicted_labels)
print("Precision: %.3f, recall: %.3f" % (training_precision, training_recall))

Precision: 0.659, recall: 0.761


Self-test: in spam classification, users probably don't mind the occasional spam email in their inbox but are quite upset if a legitimate (and important) ham email gets sent to their spambox. Knowing this, which one is more important in spam classification: precision or recall?

## F1 score

It is kind of annoying to have two numbers we want to maximise -- it'd be much easier to aggregate precision and recall into a single number that we could track when training our models. This is what [F1 score](https://en.wikipedia.org/wiki/F1_score) does -- it is high iff both precision and recall are high. (If we wanted to we could also use a weighted F1 score that assigns more importance to precision but we'll skip that for simplicity).

In [16]:
training_f1 = sklearn.metrics.f1_score(labels, predicted_labels)
print("F1 score: %.3f" % training_f1)

F1 score: 0.707


## Overfitting and cross-validation
Great! We've now trained a model and seen that it performs reasonably well. However, we don't really know if our model will do well on unseen data. How can we convince ourselves and the Product Manager that our spam detector is ready for production?

We say that a model [**overfits**](https://en.wikipedia.org/wiki/Overfitting) if it does well on the training data but is really bad at predicting previously unseen examples. If you're studying for an exam and just study the previous year's exam questions rather than understanding the material, you will probably do badly on the exam -- you overfit.

How can we tell if we've overfitted our spam detector? The standard way is to randomly divide our training data into two batches, for example 80% and 20%; train our model on the 80% and test it on the 20%. If the model does well on the unseen 20% we can be reasonably sure that it will do well in production.

"But what if we randomly picked the 20% so that the model was very lucky and did well on the 20% by chance?"

I'm glad you asked. This is why we use [**cross-validation**](https://en.wikipedia.org/wiki/Cross-validation_(statistics)): we repeat this 80-20 split and training and evaluation five times -- each time picking a different (non-overlapping) 20% to test our model on. (This specifically is called k-fold cross-validation and in our case, k=5).


If the above seemed complicated, you're in luck -- it's just one line with scikit-learn.

In [17]:
NUM_CV_FOLDS = 5
model_cv = LogisticRegression()
scores = cross_validation.cross_val_score(model_cv, features, labels, cv=NUM_CV_FOLDS, scoring='f1')
print("Mean F1 score in %d-fold cross-validation: %.3f" % (NUM_CV_FOLDS, np.mean(scores)))

Mean F1 score in 5-fold cross-validation: 0.701


This is definitely better than [random](http://stats.stackexchange.com/questions/43102/good-f1-score-for-anomaly-detection) -- the F1 score of randomly guessing would be 0.5 in our case.

## Improving our model

How could we improve? Some of the first things to try are:
1. Add more features.
2. Find a better model (i.e. use something other than logistic regression).
3. Find better hyperparameters for your model.

Let's try adding more features. In theory, we're only limited by our imagination. In practice I would try to find things about the problem that you have a good hunch about (that you would use if you were to manually classify spam) and convert them into numbers.

Our new feature extractor will look like this:

In [18]:
def simple_features_extended(text):
    """Extract some more simple numeric features from one email."""
    count_free = text.count("free")
    count_credit = text.count("money")
    count_penis = text.count("penis")
    count_pill = text.count("pill")
    return [count_free, count_credit, count_penis, count_pill] + simple_features(text)

Note that we're using the old features as well.

Let's calculate the dataset again...

In [19]:
features_ext, labels = get_dataset(simple_features_extended, number_of_emails=NUM_EMAILS)

Extracted features using function 'simple_features_extended'
2000 emails processed: 1000 ham, 1000 spam


...and plug them into a new LogisticRegression object.

In [20]:
model2 = LogisticRegression()
scores = cross_validation.cross_val_score(model2, features_ext, labels, cv=NUM_CV_FOLDS, scoring='f1')
print("Mean F1 score in %d-fold cross-validation: %.3f" % (NUM_CV_FOLDS, np.mean(scores)))

Mean F1 score in 5-fold cross-validation: 0.704


We're doing very slightly better but not by much. Perhaps a more complex model could help us?

**Support vector machines** or [SVMs](https://en.wikipedia.org/wiki/Support_vector_machine) were the state of the art in many tasks before deep neural networks came around. Starting a machine learning task with training an SVM is often a good idea -- they are powerful and well understood so there are a lot of ~~StackOverflow answers~~ useful resources out there.

Let's try an SVM.